# Streaming Videos Cache Optimization Problem
Using cache servers, we can optimize requests for videos from a data center to endpoints. Based on the predicted requests from endpoints, can we find a way to optimize the distribution and storage of said videos?

## Input and Parsing
Data is provided as text. We can parse said data into various tokens. We read from `data/input.txt` in this case.
The `problem_description` array holds, in order, from 0 to 4, the number of videos, number of endpoints, number of request descriptions, number of cache servers, and the capacity of each cache server in megabytes.
The `video_size` array holds the size of each video in MB.
We then parse based on the ammount of endpoints, to connect each endpoint to the caches. The `endpoint_data_description` describes the latency between an endpoint (serves as the index of the array) and the data center (latency is the value stored), and the `endpoint_cache_description` has the key/value specification of key:(endpoint, cache) -> value:latency.
Finally, `request_description` is a dictionary that holds the ammount of requests a certain video at an endpoint holds, specification of key:(endpoint, video) -> value:nº of requests.

In [25]:
problem_description = []
video_size = []
endpoint_data_description = []
endpoint_cache_description = {}
request_description = {}
with open('data/kittens.in.txt', 'r') as file:
    line = file.readline()
    tokens = line.strip().split()
    for token in tokens:
        problem_description.append(int(token))
    line = file.readline()
    tokens = line.strip().split()
    for token in tokens:
        video_size.append(int(token))
    i = 0
    while i != problem_description[1]:
        line = file.readline()
        tokens = line.strip().split()
        endpoint_data_description.append(int(tokens[0]))
        connections = int(tokens[1])
        j = 0
        while j < connections:
            line = file.readline()
            tokens = line.strip().split()
            endpoint_cache_description[(i, tokens[0])] = tokens[1]
            j += 1
        i += 1
    i = 0
    while i != problem_description[4]:
        line = file.readline()
        tokens = line.strip().split()
        print(len(tokens))
        request_description[(tokens[1], tokens[0])] = tokens[2]
print(len(request_description))

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


IndexError: list index out of range

## Goal and Scoring
Our goal is to maximize time saved by the caches, for this, we must go through our current cache configuration and figure out how much time we are saving in total based on the requests. Then, we multiply this value, in milliseconds, by 1000, to get the score. 
Time Saved (Request Description) = Nº of Requests * min(Latency of Data Center - Latency of Cache with Video)